In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import models, layers, optimizers

from keras import datasets

# Dividing the cifar100 fine data into train and test sets
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


Dataset Division: Divide the training dataset into two parts: a sub-
training set and a validation set. Allocate randomly 1
5 of the training dataset to
serve as the validation set.

In [ ]:
X_sub_train, X_val, y_sub_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
# dividing the train data into sub train and validations 0.2 randomly

Label Prediction Requirement: Ensure that your model is designed to predict the
“fine” label (class) rather than the “coarse” label (superclass). Important: Your
model must predict the “fine” label to be considered for grading; predicting the
“coarse” label will result in a score of zero

Model Selection: Using the sub-training and validation sets, identify
the three most effective models



In [ ]:

def AlexNet():
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), strides=(1,1), padding = 'same', activation='relu', input_shape=(32,32,3)),
        #input shape 32x32x3 to match the dataset, 64 filter size was chosen to
        #get an increased basic featurs, kept padding the same through out the
        #model Stride 1x1 to get every pixel relu out performed with tanh and sigmoid
        layers.Conv2D(192, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)), #added L2 regularizers to improve overfitting
        # increased filter to 192 enabled more complex features
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2),
        layers.Conv2D(384, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(384, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2), # added dropout to help wiht overfitting
        layers.Flatten(), # flatten turn 2D feature map int 1D
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(100, activation='softmax') # 100 for the multi class classification and softmax


    ])
    return model

In [ ]:
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

#adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

#sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
# three different optimizers i tried with RMSprop learning_rate=1e-4 being the best

cnn_model1 = AlexNet()
# I read sparse_categorical_crossentropy is best used in is best used for classification for reducing memory issue which i ran into
cnn_model1.compile(loss='sparse_categorical_crossentropy', optimizer = rmsprop_optimizer, metrics=['accuracy'])
#history = cnn_model1.fit(X_sub_train, y_sub_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))
history = cnn_model1.fit(X_train, y_train, epochs=10, batch_size=64)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Epoch 1/10
782/782 [==============================] - 50s 62ms/step - loss: 4.0853 - accuracy: 0.1485
Epoch 2/10
782/782 [==============================] - 48s 61ms/step - loss: 2.7123 - accuracy: 0.3386
Epoch 3/10
782/782 [==============================] - 48s 62ms/step - loss: 2.0180 - accuracy: 0.4920
Epoch 4/10
782/782 [==============================] - 48s 61ms/step - loss: 1.2807 - accuracy: 0.6646
Epoch 5/10
782/782 [==============================] - 48s 61ms/step - loss: 0.6489 - accuracy: 0.8296
Epoch 6/10
782/782 [==============================] - 48s 62ms/step - loss: 0.3651 - accuracy: 0.9107
Epoch 7/10
782/782 [==============================] - 48s 61ms/step - loss: 0.2685 - accuracy: 0.9395
Epoch 8/10
782/782 [==============================] - 48s 61ms/step - loss: 0.2218 - accuracy: 0.9541
Epoch 9/10
782/782 [==============================] - 48s 62ms/step - loss: 0.1897 - accuracy: 0.9631
Epoch 10/10
782/782 [==============================] - 48s 62ms/step - loss: 0.166

KeyError: ignored

In [ ]:
test_model1=cnn_model1.evaluate(X_test,y_test)
cnn_model1.summary()

313/313 [==============================] - 3s 9ms/step - loss: 4.2410 - accuracy: 0.4259
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_66 (Conv2D)          (None, 32, 32, 192)       110784    
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 16, 16, 192)       0         
 ng2D)                                                           
                                                                 
 dropout_31 (Dropout)        (None, 16, 16, 192)       0         
                                                                 
 conv2d_67 (Conv2D)          (None, 16, 16, 384)       663936    
                                                                 
 conv2d_68 (Conv2D)          (N

In [ ]:


def vgg(): #vgg was used as a base structure for my second model
            # it gave me great place to start with for focusing on filters
    model = models.Sequential([ #filter sizes are similar to vgg, when testing i kept running into overfitting so i added L2 regularizers
        layers.Conv2D(64, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4), input_shape=(32,32,3)),
        layers.Conv2D(64, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2), #added dropout at 0.2 to reduce overfitting and 0.2 was chosen becuase others performed worst
        layers.Conv2D(128, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(128, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        # using conv2d, conv2d, maxpool2d, dropout, i wanted to double to filters
        # from 64, 128, 256, 512 to keep increasing network complex features
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2),
        layers.Conv2D(256, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(256, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(256, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2),
        layers.Conv2D(512, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(512, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(512, (3, 3), padding = 'same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2),
        #add one more block but with tanh as an activation because i gained 2% in peformance
        layers.Conv2D(512, (3, 3), padding = 'same', activation='tanh', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(512, (3, 3), padding = 'same', activation='tanh', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(512, (3, 3), padding = 'same', activation='tanh', kernel_regularizer=regularizers.l2(1e-4)),
        layers.MaxPool2D(pool_size= (2,2), strides=(2,2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(100, activation='softmax')


    ])
    return model
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

#adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

#sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)


cnn_model2 = vgg()
cnn_model2.compile(loss='sparse_categorical_crossentropy', optimizer = rmsprop_optimizer, metrics=['accuracy'])
#history = cnn_model2.fit(X_sub_train, y_sub_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))
history = cnn_model2.fit(X_train, y_train, epochs=10, batch_size=128)
test_model2=cnn_model2.evaluate(X_test,y_test)
cnn_model2.summary()

Epoch 1/10
391/391 [==============================] - 32s 72ms/step - loss: 4.8109 - accuracy: 0.0221
Epoch 2/10
391/391 [==============================] - 26s 67ms/step - loss: 4.4395 - accuracy: 0.0584
Epoch 3/10
391/391 [==============================] - 26s 66ms/step - loss: 4.1760 - accuracy: 0.0985
Epoch 4/10
391/391 [==============================] - 26s 67ms/step - loss: 3.9236 - accuracy: 0.1438
Epoch 5/10
391/391 [==============================] - 26s 67ms/step - loss: 3.7134 - accuracy: 0.1760
Epoch 6/10
391/391 [==============================] - 26s 67ms/step - loss: 3.5408 - accuracy: 0.2080
Epoch 7/10
391/391 [==============================] - 26s 67ms/step - loss: 3.3671 - accuracy: 0.2414
Epoch 8/10
391/391 [==============================] - 26s 67ms/step - loss: 3.2219 - accuracy: 0.2699
Epoch 9/10
391/391 [==============================] - 26s 67ms/step - loss: 3.0724 - accuracy: 0.2966
Epoch 10/10
313/313 [==============================] - 3s 9ms/step - loss: 2.9681 

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:

def create_model3(): # this model was created from what i learned from the previous two models
                    # i noticed L2 regularizers decreased overfitting
                    # adding BatchNormalization help with my issue of training plateaus
                    # kept relu as the activation as it peformed best
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=(32, 32, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=regularizers.l2(1e-4), activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(100, activation='softmax')
    ])
    return model
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

#adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

#sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
cnn_model3 = create_model3()
cnn_model3.compile(loss='sparse_categorical_crossentropy', optimizer = rmsprop_optimizer, metrics=['accuracy'])
#history = cnn_model3.fit(X_sub_train, y_sub_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))
history = cnn_model3.fit(X_train, y_train, epochs=10, batch_size=128)
test_model3=cnn_model3.evaluate(X_test,y_test)
cnn_model3.summary()

Epoch 1/10
391/391 [==============================] - 17s 35ms/step - loss: 4.3940 - accuracy: 0.0794
Epoch 2/10
391/391 [==============================] - 13s 34ms/step - loss: 3.7759 - accuracy: 0.1615
Epoch 3/10
391/391 [==============================] - 13s 34ms/step - loss: 3.4500 - accuracy: 0.2138
Epoch 4/10
391/391 [==============================] - 13s 34ms/step - loss: 3.1852 - accuracy: 0.2647
Epoch 5/10
391/391 [==============================] - 13s 34ms/step - loss: 2.9723 - accuracy: 0.3008
Epoch 6/10
391/391 [==============================] - 13s 34ms/step - loss: 2.7907 - accuracy: 0.3374
Epoch 7/10
391/391 [==============================] - 13s 34ms/step - loss: 2.6294 - accuracy: 0.3671
Epoch 8/10
391/391 [==============================] - 13s 34ms/step - loss: 2.4998 - accuracy: 0.3943
Epoch 9/10
391/391 [==============================] - 13s 34ms/step - loss: 2.3822 - accuracy: 0.4216
Epoch 10/10
313/313 [==============================] - 2s 6ms/step - loss: 2.3846 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()